# 1 SCRAPPING THE WIKIPEDIA WEB PAGE
## importing libraries

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [2]:
site= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(site).text

In [3]:
soup = BeautifulSoup(page,"html5lib")

## finding right table by run a for loop in search of neigbourhoods borough

In [4]:
tables = soup.find_all("table")
type(tables)

bs4.element.ResultSet

### create dataframe

In [5]:
df_hood= pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])

In [6]:
row1=tables[0].tbody.find_all('td')
type(row1)
#row1[4].text.replace('/','and')
#col1=row1.find_all('b')
#col1
# TESTING CELL

bs4.element.ResultSet

### using a for loop through the entire table and appending result to dataframe result in below
    

In [7]:
for col in row1:
    if ("Not assigned") not in col.text:
        ab=col.text
        code= ab[1:4]
        hood=ab.split('(')[1][:-3]
        hood=hood.replace('/',', ')
        borough=ab.split('(')[0][4:]
        df_hood=df_hood.append({'Postal Code':code,'Borough':borough,'Neighborhood':hood}, ignore_index=True)
df_hood.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
df_hood.shape

(103, 3)

### installing geocoder for the first time

In [ ]:
pip install geocoder

### using the geocoder package to get latitiude and longitude

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code="M5A"# TESTING CODE WITH PARKWOODS POSTALCODE

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude,longitude)

### reverting to geodate CSV file

In [ ]:
import os
cwd = os.getcwd()
cwd

In [9]:
geodata=pd.read_csv("Geospatial_Coordinates.csv")
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


df_h=df_hood
df_h[['latitude']]=geodata['Latitude'](df_h['Postal Code'] == geodata['Postal Code'])
df_h.head()

In [11]:
df_hood_cord =df_hood.join(geodata.set_index('Postal Code'),on='Postal Code')
df_hood_cord.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills)Nort,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
